# Tide Analysis demonstration

See the `analysea` [README](https://github.com/tomsail/analysea) for more information


In [ ]:
from analysea import TideAnalysisResults
import pandas as pd
import numpy as np

## load test case : Aberdeen

In [ ]:
_ioc_station = "abed" 
df = pd.read_parquet(f"../tests/data/{_ioc_station}.parquet")
df.describe()

In [ ]:
df.plot()

## Live detide and reconstruct

In [ ]:
opts = {'conf_int': 'linear',
        'constit' : 'auto',
        'method' : 'ols', # ols is faster and good for missing data (Ponchaut et al., 2001)
        'order_constit' : 'frequency',
        'Rayleigh_min' : 0.97,
        'lat': 57.14,
        'verbose' : True,
} # careful if there is only one Nan parameter, the analysis crashes

In [ ]:
from analysea.tide import tide_analysis
ta = tide_analysis(df["bub"], resample_detide=True, **opts) 

Attention: Resampling allows faster analysis but it omits data

see `tide_analysis` function documentation to know more about resampling options (by default deactivated)

In [ ]:
ta.surge.plot()

# Analyse tide consituents

In [ ]:
# main constituents 
ASTRO_PLOT = [ "M2", "S2", "N2", "O1", "K2", "K1", "NU2", "Q1", "L2", 
              "P1", "2N2", "M4", "MS4", "MM", "MU2", "SSA", "LDA2", "MF", "MSM", "MN4"]

constituents found by Utide: 

In [ ]:
ta.coefs["name"]

In [ ]:
def subset_tidal_analysis(ta: TideAnalysisResults, astro_coefs: list[str]
                          )-> np.array:
    def get_tidal_const_index(utide_list: list[str], astro_coefs: list[str]):
        index = []
        for ast in astro_coefs : 
            try :
                index.extend([utide_list.index(ast)])
            except ValueError as err: 
                index.append(float('nan'))
                print('[WARN]',err)
        return index

    astro_coefs = [ast.upper() for ast in astro_coefs]
    if isinstance(ta.coefs, list):
        amps_coef = []
        phas_coef = []
        for iy, year in enumerate(ta.years):
            utide_list = list(ta.coefs[iy]["name"])
            index = get_tidal_const_index(utide_list, astro_coefs)
            amps_coef.append(ta.coefs[iy]['A'][index])
            phas_coef.append(ta.coefs[iy]['g'][index])
        return amps_coef, phas_coef

    elif isinstance(ta.coefs, dict):
        utide_list = list(ta.coefs["name"])
        index = get_tidal_const_index(utide_list, astro_coefs)
        amps_coef = ta.coefs['A'][index]
        phas_coef = ta.coefs['g'][index]
        return amps_coef, phas_coef
    else: 
        raise ValueError(f"Unknown format {ta.coefs}")

amps, phas = subset_tidal_analysis(ta, ASTRO_PLOT)

In [ ]:
pd_coefs = pd.DataFrame({"constituents": ASTRO_PLOT, "Amplitude all": amps, "Phase all": phas})
pd_coefs.plot.bar(x="constituents", y="Amplitude all")
pd_coefs.plot.bar(x="constituents", y="Phase all")

## compare with multiyear tide analysis

In [ ]:
from analysea.tide import yearly_tide_analysis
ta_multi = yearly_tide_analysis(df["bub"], resample_detide=True, **opts) 

In [ ]:
ta_multi.years

In [ ]:
amps_multi, phas_multi = subset_tidal_analysis(ta_multi, ASTRO_PLOT)
pd_coefs_multi = pd.DataFrame()
for iy, year in enumerate(ta_multi.years):
    pd_coefs_ = pd.DataFrame({
        f"Amplitude {year}": amps_multi[iy], 
        f"Phase {year}": phas_multi[iy]})
    pd_coefs_multi = pd.concat([pd_coefs_multi, pd_coefs_], axis=1)

color={"Amplitude all": "black", 
       "Amplitude 2022": "green",
       "Amplitude 2023": "blue",
       "Phase all": "black", 
       "Phase 2022": "green",
       "Phase 2023": "blue"}
pd.concat([pd_coefs, pd_coefs_multi], axis = 1).plot(
    x="constituents", 
    y=["Amplitude all"] + [f"Amplitude {year}" for year in ta_multi.years], 
    kind="bar",
    color=color)
pd.concat([pd_coefs, pd_coefs_multi], axis = 1).plot(
    x="constituents", 
    y=["Phase all"] + [f"Phase {year}" for year in ta_multi.years], 
    kind="bar",
    color=color)